# Обработка данных

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [6]:
# 0. Загрузка и подготовка

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import numpy as np
import pyarrow.dataset as ds

# Пути к файлам
import glob
parquet_path = glob.glob("/content/drive/MyDrive/*/transaction_fraud_data.parquet")[0]
fx_path = glob.glob("/content/drive/MyDrive/*/historical-currency-exchange.csv")[0]

# Чтение parquet
dataset = ds.dataset(parquet_path, format="parquet")

# Чтение CSV с курсами валют
fx = pd.read_csv(fx_path)
fx["date"] = pd.to_datetime(fx["date"], errors="coerce").dt.date
fx_long = fx.melt(id_vars="date", var_name="currency", value_name="rate").dropna(subset=["rate"])
fx_long["currency"] = fx_long["currency"].str.upper()
fx_map = {(d, c): float(r) for d, c, r in fx_long.itertuples(index=False)}

print(f"Файл транзакций: {parquet_path}")
print(f"Файл курсов валют: {fx_path}")


Mounted at /content/drive
Файл транзакций: /content/drive/MyDrive/ИТМО/transaction_fraud_data.parquet
Файл курсов валют: /content/drive/MyDrive/ИТМО/historical-currency-exchange.csv


In [11]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.4 MB/s eta 0:00:00


# Формирование fraud_quick_audit_report.xlsx

In [13]:
import xlsxwriter
import os, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Путь для сохранения одного Excel-файла
OUT_XLSX = Path("/content/drive/MyDrive/ИТМО/fraud_quick_audit_report.xlsx")

def to_pandas_head(dataset, n):
    if hasattr(dataset, "head") and hasattr(dataset.head(1), "to_pandas"):
        return dataset.head(n).to_pandas()
    if isinstance(dataset, pd.DataFrame):
        return dataset.head(n).copy()
    raise TypeError("dataset не распознан")

def safe_nunique(s: pd.Series) -> int:
    return s.map(lambda x: json.dumps(x, sort_keys=True) if isinstance(x, (dict, list)) else x) \
            .nunique(dropna=True)

def parse_lha_val(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    try:
        return json.loads(x)
    except Exception:
        return {}

def ensure_datetime(s):
    return pd.to_datetime(s, errors="coerce", utc=True)

def run_combined_report(dataset):
    with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as writer:
        # 1) Быстрый профиль
        df = to_pandas_head(dataset, 100_000)
        desc = df.describe(include=[np.number]).T
        desc.to_excel(writer, sheet_name="profile_numeric")

        na = df.isna().sum().sort_values(ascending=False).to_frame(name="na_count")
        na["na_rate"] = na["na_count"] / len(df)
        na.to_excel(writer, sheet_name="profile_na")

        uniq = pd.Series({col: safe_nunique(df[col]) for col in df.columns}, name="nunique_safe")
        uniq.sort_values(ascending=False).to_frame().to_excel(writer, sheet_name="profile_nunique")

        if "last_hour_activity" in df.columns:
            keys = ["num_transactions", "total_amount", "unique_merchants", "unique_countries", "max_single_amount"]
            lha = df["last_hour_activity"].map(parse_lha_val)
            rows = []
            for k in keys:
                vals = pd.to_numeric(lha.map(lambda d: d.get(k, np.nan)), errors="coerce")
                rows.append({
                    "key": k,
                    "non_null": int(vals.notna().sum()),
                    "min": float(vals.min()) if vals.notna().any() else np.nan,
                    "mean": float(vals.mean()) if vals.notna().any() else np.nan,
                    "max": float(vals.max()) if vals.notna().any() else np.nan,
                })
            pd.DataFrame(rows).to_excel(writer, sheet_name="profile_last_hour_activity", index=False)

        df.head(20).to_excel(writer, sheet_name="sample_head", index=False)

        # 2) Лифты
        df_big = to_pandas_head(dataset, 1_000_000)
        base = df_big["is_fraud"].mean()

        def lift_one(col):
            t = df_big.groupby(col)["is_fraud"].agg(['mean','count']).rename(columns={'mean':'fraud_rate','count':'transactions'})
            t["lift"] = t["fraud_rate"] / base
            return t.sort_values("lift", ascending=False)

        for col in ["vendor_category", "vendor_type"]:
            if col in df_big.columns:
                lift_one(col).to_excel(writer, sheet_name=f"lift_{col}")

        def lift_table(group_cols, min_n=300):
            t = (df_big.groupby(group_cols, dropna=False)["is_fraud"]
                 .agg(['mean','count'])
                 .rename(columns={'mean':'fraud_rate','count':'transactions'})
                 .reset_index())
            t["lift"] = t["fraud_rate"] / base
            return t[t["transactions"] >= min_n].sort_values("lift", ascending=False)

        combos = [
            (["channel","device"], 300, "lift_channel_x_device"),
            (["is_outside_home_country","is_high_risk_vendor"], 200, "lift_outside_x_highrisk"),
            (["country"], 500, "lift_country"),
            (["country","city"], 400, "lift_country_x_city"),
        ]
        for cols, min_n, name in combos:
            if set(cols).issubset(df_big.columns):
                lift_table(cols, min_n).to_excel(writer, sheet_name=name, index=False)

        # 3) Время
        if "timestamp" in df_big.columns:
            if not np.issubdtype(df_big["timestamp"].dtype, np.datetime64):
                df_big["timestamp"] = ensure_datetime(df_big["timestamp"])
            df_big["day_of_week"] = df_big["timestamp"].dt.day_name()
            df_big["hour"] = df_big["timestamp"].dt.hour
            pivot = df_big.pivot_table(index="day_of_week", columns="hour",
                                       values="is_fraud", aggfunc="mean").fillna(0.0)
            days_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
            pivot = pivot.reindex(days_order)
            pivot.to_excel(writer, sheet_name="fraud_rate_by_dow_hour")

        # 4) Кандидаты на фичи
        if "timestamp" in df.columns:
            df["timestamp"] = ensure_datetime(df["timestamp"])
        if set(["country","city","vendor_type","amount"]).issubset(df.columns):
            med = df.groupby(["country","city","vendor_type"])["amount"].median().rename("city_vendor_median")
            df = df.merge(med, on=["country","city","vendor_type"], how="left")
            df["RelativeAmountCity"] = df["amount"] / df["city_vendor_median"].replace(0, np.nan)
            q = df["RelativeAmountCity"].quantile(0.995)
            df["RelativeAmountCity"] = df["RelativeAmountCity"].clip(upper=q)

        if "is_fraud" in df.columns:
            df["hour"] = df["timestamp"].dt.hour
            hr = df.groupby("hour")["is_fraud"].mean().rename("HourRiskMap")
            df = df.merge(hr, on="hour", how="left")
            df["HourRisk"] = df["HourRiskMap"]

        if set(["customer_id","device_fingerprint","timestamp"]).issubset(df.columns):
            first_ts = df.groupby(["customer_id","device_fingerprint"])["timestamp"].transform("min")
            df["DeviceFirstSeen"] = (df["timestamp"] == first_ts).astype("int8")

        if "last_hour_activity" in df.columns:
            lha = df["last_hour_activity"].map(parse_lha_val)
            df["lha_num_tx"] = pd.to_numeric(lha.map(lambda d: d.get("num_transactions", np.nan)), errors="coerce")
            df["lha_total_amount"] = pd.to_numeric(lha.map(lambda d: d.get("total_amount", np.nan)), errors="coerce")
            z_tx = (df["lha_num_tx"] - df["lha_num_tx"].mean()) / (df["lha_num_tx"].std(ddof=0) + 1e-9)
            z_amt = (np.log1p(df["lha_total_amount"]) - np.log1p(df["lha_total_amount"]).mean()) / (np.log1p(df["lha_total_amount"]).std(ddof=0) + 1e-9)
            df["HyperActivityScore"] = (z_tx + z_amt).clip(-5, 5)

        cand = [c for c in ["RelativeAmountCity","HourRisk","DeviceFirstSeen","HyperActivityScore"] if c in df.columns]
        if cand and "is_fraud" in df.columns:
            corr = df[cand].join(df["is_fraud"].astype(int)).corr(method="spearman").loc[cand, "is_fraud"] \
                .sort_values(ascending=False).to_frame("spearman_vs_fraud")
            corr.to_excel(writer, sheet_name="feature_candidates_spearman")
            df[cand].quantile([0.01,0.5,0.99]).to_excel(writer, sheet_name="feature_candidates_quantiles")
            keep = ["transaction_id","is_fraud"] + cand
            df[keep].to_excel(writer, sheet_name="feature_candidates", index=False)

    print(f"Отчёт сохранён в: {OUT_XLSX}")

# ---- Запуск ----
run_combined_report(dataset)

Отчёт сохранён в: /content/drive/MyDrive/ИТМО/fraud_quick_audit_report.xlsx


# Проверка 10 гипотез

In [1]:
# === ГИПОТЕЗЫ: экономная загрузка + конвертация валют + проверка 10/10 ===
import os, json, math, gc
from pathlib import Path
import numpy as np
import pandas as pd

# ---------- ПУТИ ----------
PARQUET_PATH = "/content/drive/MyDrive/ИТМО/transaction_fraud_data.parquet"   # ваш паркет
FX_CSV_PATH  = "/content/drive/MyDrive/ИТМО/historical-currency-exchange.csv" # CSV с курсами
OUT_DIR      = Path("/content/drive/MyDrive/ИТМО/audit_out"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------- КОНФИГ ПРАГМАТИКИ / ПОРОГОВ ----------
MIN_N       = 300       # минимальный надёжный объём сегмента
LIFT_THR    = 1.20      # «сильный» лифт
FR_THR      = 0.15      # «сильная» доля фрода (для фич, где считаем пороги)
ZSIGMA_HA   = 2.0       # порог для HyperActivityScore (z>2)
RELAMT_Q    = 0.95      # «высокая сумма» — квантиль по amount_usd
LHA_COUNTRY = 3         # «> N уникальных стран» за час
LHA_MERCH   = 5         # «> N уникальных мерчантов» за час

# ---------- КОЛОНКИ, которые тянем из паркета (экономно) ----------
COLS = [
    "transaction_id","customer_id","device_fingerprint",
    "timestamp","is_fraud",
    "vendor_category","vendor_type",
    "channel","device",
    "country","city",
    "is_outside_home_country","is_high_risk_vendor",
    "amount","currency",
    "last_hour_activity"
]

# ---------- УТИЛИТЫ ----------
def to_datetime_utc(s):
    s = pd.to_datetime(s, errors="coerce", utc=True)
    return s

def parse_lha(x):
    if isinstance(x, dict): return x
    if pd.isna(x): return {}
    try:
        return json.loads(x)
    except Exception:
        return {}

def safe_rate_lookup(row, fx_wide):
    # row: has date_floor, currency; fx_wide has columns ['date','USD','EUR',...], USD=1
    # если валюты нет в fx или NaN — вернём NaN => amount_usd станет NaN (мы их отбросим в соответствующих гипотезах)
    try:
        return fx_wide.at[row["date_floor"], row["currency"]]
    except Exception:
        return np.nan

def segment_lift(df, seg_mask, min_n=MIN_N):
    """Лифт и показатели для булевого сегмента."""
    n_all = df["is_fraud"].notna().sum()
    base = df["is_fraud"].mean() if n_all else np.nan

    seg = df.loc[seg_mask]
    n = seg["is_fraud"].notna().sum()
    fr = seg["is_fraud"].mean() if n else np.nan
    lift = (fr / base) if (n and base and base>0) else np.nan
    return {
        "n": int(n),
        "fraud_rate": float(fr) if pd.notna(fr) else np.nan,
        "lift": float(lift) if pd.notna(lift) else np.nan
    }

def top_lift_by_group(df, cols, min_n=MIN_N, top_k=1):
    base = df["is_fraud"].mean()
    g = (df.groupby(cols, dropna=False)["is_fraud"]
           .agg(['mean','count'])
           .rename(columns={'mean':'fraud_rate','count':'n'})
           .reset_index())
    g = g[g["n"]>=min_n]
    g["lift"] = g["fraud_rate"] / base
    g = g.sort_values("lift", ascending=False)
    if g.empty:
        return np.nan, None
    row = g.iloc[0:top_k]
    return float(row["lift"].iloc[0]), row

def print_line(name, ok, metric, extra=""):
    badge = "✅" if ok else "⚠️"
    print(f"{badge} {name:35s} | passed={str(ok):5s} | metric={metric:<8.3f} {extra}")

# ---------- 1) ЗАГРУЗКА ДАННЫХ (экономно) ----------
try:
    df = pd.read_parquet(PARQUET_PATH, columns=COLS)
except MemoryError:
    # fallback: по частям через pyarrow.dataset (если совсем тесно по памяти)
    import pyarrow.dataset as ds
    dataset = ds.dataset(PARQUET_PATH, format="parquet")
    batches = []
    for b in dataset.to_batches(columns=COLS, batch_size=200_000):
        batches.append(b.to_pandas())
    df = pd.concat(batches, ignore_index=True)
    del batches; gc.collect()

# только строки с is_fraud в [0,1]
df = df[df["is_fraud"].isin([0,1])].copy()
df["is_fraud"] = df["is_fraud"].astype("float32")

# ---------- 2) КОНВЕРТАЦИЯ ВАЛЮТ ----------
fx = pd.read_csv(FX_CSV_PATH)
# приводим к «широкому» виду: индекс date, колонки — коды валют, значения — курс (единиц валюты за 1 USD)
fx["date"] = pd.to_datetime(fx["date"], utc=True)
fx = fx.set_index("date").sort_index()
df["timestamp"] = to_datetime_utc(df["timestamp"])
df["date_floor"] = df["timestamp"].dt.floor("D")

# убедимся, что у нас есть только нужные даты
fx_wide = fx.copy()

# векторный маппинг курса
# подготовим возможные пропуски/редкие валюты
avail_curr = set(fx_wide.columns)
mask_known = df["currency"].isin(avail_curr)
df["fx_rate"] = np.nan
if mask_known.any():
    # построчный lookup (быстро на Cython не сделать — но этот участок обычно не узкое место)
    df.loc[mask_known, "fx_rate"] = df.loc[mask_known, ["date_floor","currency"]].apply(
        lambda r: fx_wide.at[r["date_floor"], r["currency"]]
        if (r["date_floor"] in fx_wide.index) and (r["currency"] in fx_wide.columns)
        else np.nan, axis=1
    )

# amount_usd: amount / rate (т.к. курсы «за 1 USD»)
df["amount_usd"] = df["amount"] / df["fx_rate"]
# невалидные — в NaN
df.loc[~np.isfinite(df["amount_usd"]), "amount_usd"] = np.nan

# ---------- 3) БАЗОВЫЕ ВЫЧИСЛЕНИЯ ----------
base_fr = df["is_fraud"].mean()
base_n  = len(df)
summary_rows = []

# ===== Гипотеза 1: Лифты по vendor_category =====
ok1_lift, top1 = top_lift_by_group(df.dropna(subset=["vendor_category"]), ["vendor_category"], MIN_N)
summary_rows.append(["lift_vendor_category", ok1_lift>=LIFT_THR if pd.notna(ok1_lift) else False, ok1_lift, int(top1["n"].iloc[0]) if isinstance(top1, pd.DataFrame) else 0])

# ===== Гипотеза 2: Лифты по vendor_type =====
ok2_lift, top2 = top_lift_by_group(df.dropna(subset=["vendor_type"]), ["vendor_type"], MIN_N)
summary_rows.append(["lift_vendor_type", ok2_lift>=LIFT_THR if pd.notna(ok2_lift) else False, ok2_lift, int(top2["n"].iloc[0]) if isinstance(top2, pd.DataFrame) else 0])

# ===== Гипотеза 3: Канал × устройство =====
ok3_lift, top3 = top_lift_by_group(df.dropna(subset=["channel","device"]), ["channel","device"], MIN_N)
summary_rows.append(["lift_channel_x_device", ok3_lift>=LIFT_THR if pd.notna(ok3_lift) else False, ok3_lift, int(top3["n"].iloc[0]) if isinstance(top3, pd.DataFrame) else 0])

# ===== Гипотеза 4: Гео страна =====
ok4_lift, top4 = top_lift_by_group(df.dropna(subset=["country"]), ["country"], MIN_N)
summary_rows.append(["lift_country", ok4_lift>=LIFT_THR if pd.notna(ok4_lift) else False, ok4_lift, int(top4["n"].iloc[0]) if isinstance(top4, pd.DataFrame) else 0])

# ===== Гипотеза 5: Гео страна×город =====
ok5_lift, top5 = top_lift_by_group(df.dropna(subset=["country","city"]), ["country","city"], MIN_N)
summary_rows.append(["lift_country_x_city", ok5_lift>=LIFT_THR if pd.notna(ok5_lift) else False, ok5_lift, int(top5["n"].iloc[0]) if isinstance(top5, pd.DataFrame) else 0])

# ===== Гипотеза 6: Outside_home × High_risk_vendor =====
ok6_lift, top6 = top_lift_by_group(df.dropna(subset=["is_outside_home_country","is_high_risk_vendor"]),
                                   ["is_outside_home_country","is_high_risk_vendor"], MIN_N)
summary_rows.append(["outside_home_x_highrisk", ok6_lift>=LIFT_THR if pd.notna(ok6_lift) else False, ok6_lift, int(top6["n"].iloc[0]) if isinstance(top6, pd.DataFrame) else 0])

# ===== Гипотеза 7: Время — fraud rate по часу (берём максимальный час) =====
dft = df.dropna(subset=["timestamp"]).copy()
dft["hour"] = dft["timestamp"].dt.hour
fr_by_hour = dft.groupby("hour")["is_fraud"].agg(['mean','count']).rename(columns={'mean':'fraud_rate','count':'n'})
fr_by_hour = fr_by_hour[fr_by_hour["n"]>=MIN_N]
max_fr = fr_by_hour["fraud_rate"].max() if not fr_by_hour.empty else np.nan
summary_rows.append(["fraud_rate_by_hour", (max_fr>=FR_THR) if pd.notna(max_fr) else False, max_fr, int(fr_by_hour["n"].max()) if not fr_by_hour.empty else 0])

# ===== Гипотеза 8: RelativeAmountCity (на USD) =====
rel_df = df.dropna(subset=["amount_usd","country","city","vendor_type"]).copy()
if not rel_df.empty:
    med = rel_df.groupby(["country","city","vendor_type"])["amount_usd"].median().rename("city_vendor_median")
    rel_df = rel_df.join(med, on=["country","city","vendor_type"])
    rel_df["RelativeAmountCity"] = rel_df["amount_usd"] / rel_df["city_vendor_median"].replace(0,np.nan)
    high_thr = rel_df["RelativeAmountCity"].quantile(RELAMT_Q)
    seg = rel_df["RelativeAmountCity"] >= high_thr
    m8 = segment_lift(rel_df, seg)
    ok8 = (m8["n"]>=MIN_N) and ((m8["lift"]>=LIFT_THR) or (m8["fraud_rate"]>=FR_THR))
else:
    m8 = {"n":0,"fraud_rate":np.nan,"lift":np.nan}; ok8=False
summary_rows.append(["relative_amount_city_high", ok8, m8["lift"], m8["n"]])

# ===== Гипотеза 9: DeviceFirstSeen × amount (high) =====
dfs_df = df.dropna(subset=["timestamp","customer_id","device_fingerprint","amount_usd"]).copy()
if not dfs_df.empty:
    first_ts = dfs_df.groupby(["customer_id","device_fingerprint"])["timestamp"].transform("min")
    dfs_df["DeviceFirstSeen"] = (dfs_df["timestamp"]==first_ts)
    amt_thr = dfs_df["amount_usd"].quantile(RELAMT_Q)
    seg = (dfs_df["DeviceFirstSeen"]) & (dfs_df["amount_usd"]>=amt_thr)
    m9 = segment_lift(dfs_df, seg)
    ok9 = (m9["n"]>=MIN_N) and ((m9["lift"]>=LIFT_THR) or (m9["fraud_rate"]>=FR_THR))
else:
    m9 = {"n":0,"fraud_rate":np.nan,"lift":np.nan}; ok9=False
summary_rows.append(["device_first_seen_x_amount", ok9, m9["lift"], m9["n"]])

# ===== Гипотеза 10: HyperActivityScore + last_hour_activity raw =====
lha_df = df.dropna(subset=["last_hour_activity"]).copy()
if not lha_df.empty:
    lha = lha_df["last_hour_activity"].map(parse_lha)
    lha_df["lha_num_tx"] = pd.to_numeric(lha.map(lambda d: d.get("num_transactions", np.nan)), errors="coerce")
    lha_df["lha_total_amount"] = pd.to_numeric(lha.map(lambda d: d.get("total_amount", np.nan)), errors="coerce")
    lha_df["lha_unique_merchants"] = pd.to_numeric(lha.map(lambda d: d.get("unique_merchants", np.nan)), errors="coerce")
    lha_df["lha_unique_countries"] = pd.to_numeric(lha.map(lambda d: d.get("unique_countries", np.nan)), errors="coerce")

    z_tx  = (lha_df["lha_num_tx"] - lha_df["lha_num_tx"].mean()) / (lha_df["lha_num_tx"].std(ddof=0) + 1e-9)
    z_amt = (np.log1p(lha_df["lha_total_amount"]) - np.log1p(lha_df["lha_total_amount"]).mean()) / (np.log1p(lha_df["lha_total_amount"]).std(ddof=0) + 1e-9)
    lha_df["HyperActivityScore"] = (z_tx + z_amt).clip(-5,5)

    # 10a: HyperActivityScore > 2σ
    seg_a = lha_df["HyperActivityScore"] > ZSIGMA_HA
    m10a = segment_lift(lha_df, seg_a)
    ok10a = (m10a["n"]>=MIN_N) and ((m10a["lift"]>=LIFT_THR) or (m10a["fraud_rate"]>=FR_THR))

    # 10b: raw-условие: много стран или много мерчантов за час
    seg_b = (lha_df["lha_unique_countries"]>LHA_COUNTRY) | (lha_df["lha_unique_merchants"]>LHA_MERCH)
    m10b = segment_lift(lha_df, seg_b)
    ok10b = (m10b["n"]>=MIN_N) and ((m10b["lift"]>=LIFT_THR) or (m10b["fraud_rate"]>=FR_THR))
else:
    m10a={"n":0,"fraud_rate":np.nan,"lift":np.nan}; ok10a=False
    m10b={"n":0,"fraud_rate":np.nan,"lift":np.nan}; ok10b=False

summary_rows.append(["hyper_activity_score>2σ", ok10a, m10a["lift"], m10a["n"]])
summary_rows.append(["lha_raw_anomaly",       ok10b, m10b["lift"], m10b["n"]])

# ---------- 4) СВОДНЫЙ ОТЧЁТ ----------
summary = pd.DataFrame(summary_rows, columns=["Hypothesis","Passed","Metric_Lift_or_FR","N"])
summary.insert(1, "Thresholds", f"LIFT_THR={LIFT_THR}; FR_THR={FR_THR}; MIN_N={MIN_N}")
summary.to_csv(OUT_DIR/"hypotheses_summary.csv", index=False)

print(f"\nВсего строк: {base_n:,}; базовая доля фрода: {base_fr:.4f}")
print("—"*88)
for _,r in summary.iterrows():
    print_line(r["Hypothesis"], r["Passed"], r["Metric_Lift_or_FR"], extra=f"| n={int(r['N'])}")

# Отдельно сохраняем топ-группы для ключевых гипотез (если пригодится для слайдов)
extras = {}
if isinstance(top3, pd.DataFrame): extras["top_channel_x_device.csv"]      = top3
if isinstance(top4, pd.DataFrame): extras["top_country.csv"]               = top4
if isinstance(top5, pd.DataFrame): extras["top_country_x_city.csv"]        = top5
if isinstance(top6, pd.DataFrame): extras["top_outside_x_highrisk.csv"]    = top6

for name, tbl in extras.items():
    tbl.to_csv(OUT_DIR/name, index=False)

print("\n[saved]", OUT_DIR/"hypotheses_summary.csv")
for k in extras: print("[saved]", OUT_DIR/k)



Всего строк: 7,483,766; базовая доля фрода: 0.1997
————————————————————————————————————————————————————————————————————————————————————————
⚠️ lift_vendor_category                | passed=False | metric=1.003    | n=935790
⚠️ lift_vendor_type                    | passed=False | metric=1.007    | n=233977
✅ lift_channel_x_device               | passed=True  | metric=5.007    | n=217324
✅ lift_country                        | passed=True  | metric=1.904    | n=785704
✅ lift_country_x_city                 | passed=True  | metric=1.904    | n=785704
✅ outside_home_x_highrisk             | passed=True  | metric=2.843    | n=1806158
✅ fraud_rate_by_hour                  | passed=True  | metric=0.593    | n=456393
✅ relative_amount_city_high           | passed=True  | metric=2.677    | n=374189
✅ device_first_seen_x_amount          | passed=True  | metric=5.001    | n=233155
✅ hyper_activity_score>2σ             | passed=True  | metric=1.016    | n=970402
✅ lha_raw_anomaly                   

In [40]:
# ---------- БЛОК 1. ЗАГРУЗКА ----------
import pandas as pd, numpy as np, textwrap, math
import matplotlib.pyplot as plt
from pathlib import Path

CSV_PATH = "/content/drive/MyDrive/ИТМО/insights/top_signals_reliable.csv"
OUT_DIR  = Path("/content/drive/MyDrive/ИТМО/insights/visuals_final")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Порогы
MIN_N   = 200     # минимальный объём, чтобы считать сигнал "надёжным"
LOW_N   = 1000    # пометка (low N) на графике
TOP_N   = 10
LIFT_THR= 1.2
FR_THR  = 0.15
SP_THR  = 0.30

df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]

# Какие колонки не включать в подпись фичи
EXCLUDE = {
    "lift","fraud_rate","spearman_vs_fraud",
    "transactions","count","metric","sheet","priority_mark",
    "mean_metric_all","index","Unnamed: 0"
}

def wrap(s, w=42):
    return textwrap.fill(str(s), width=w, break_long_words=False, replace_whitespace=False)

def make_feature_label(row: pd.Series, width=42) -> str:
    parts = []
    for col, val in row.items():
        if col in EXCLUDE or pd.isna(val):
            continue
        sval = str(int(val)) if isinstance(val,(int,float)) and float(val).is_integer() else str(val)
        if sval == "" or sval.lower()=="nan":
            continue
        parts.append(f"{col}={sval}")
    txt = " | ".join(parts) if parts else "(misc)"
    return wrap(txt, width)

def is_priority(label: str) -> bool:
    s = label.lower()
    return any([
        ("is_outside_home_country=1" in s and "is_high_risk_vendor=1" in s),
        "devicefirstseen" in s,
        "hyperactivityscore" in s,
        "relativeamountcity" in s,
        ("channel=" in s and "device=" in s)
    ])

# Служебные удобства
def pick_weight_col(frame: pd.DataFrame):
    return "transactions" if "transactions" in frame.columns else ("count" if "count" in frame.columns else None)

print("Файл:", CSV_PATH, "| строк:", len(df))


Файл: /content/drive/MyDrive/ИТМО/insights/top_signals_reliable.csv | строк: 48


# Визуализация лучших гипотез

In [41]:
# ---------- БЛОК 2 (V2). ПРЕЗЕНТАЦИОННЫЕ СЛАЙДЫ: ГЕО / ПОВЕДЕНИЕ / ТРАНЗАКЦИИ ----------
import numpy as np, pandas as pd, re, os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# ВХОД: df = pd.read_csv("/content/drive/MyDrive/ИТМО/insights/top_signals_reliable.csv")
# (или из Excel-пайплайна), плюс хелперы/константы из блока 1 уже определены.

empty_notes = []
pdf_path = OUT_DIR / "slides_final_v2.pdf"
pdf = PdfPages(pdf_path)

# ===== 1) ФИЛЬТР СИЛЬНЫХ СИГНАЛОВ =====
def strong_filter(frame: pd.DataFrame) -> pd.DataFrame:
    if frame.empty:
        return frame.copy()
    m = np.zeros(len(frame), dtype=bool)
    if "lift" in frame:               m |= frame["lift"].fillna(0) >= LIFT_THR
    if "fraud_rate" in frame:         m |= frame["fraud_rate"].fillna(0) >= FR_THR
    if "spearman_vs_fraud" in frame:  m |= frame["spearman_vs_fraud"].abs().fillna(0) >= SP_THR
    w = pick_weight_col(frame)
    if w: m &= frame[w].fillna(0) >= MIN_N
    return frame[m].copy()

strong = strong_filter(df)
if strong.empty:
    raise SystemExit("После фильтров сильных сигналов нет. Проверь пороги/путь к файлам.")

# Читаемые ярлыки + приоритеты
strong["feature"]  = strong.apply(make_feature_label, axis=1)
strong["priority"] = strong["feature"].apply(is_priority)
weight_col = pick_weight_col(strong)

# ===== 2) КЛАССИФИКАЦИЯ СИГНАЛОВ НА БЛОКИ =====
def classify_block(txt: str) -> str:
    s = txt.lower()
    if "country=" in s or "city=" in s:
        return "geo"
    if any(k in s for k in [
        "device=", "devicefirstseen", "hyperactivityscore", "relativeamountcity",
        "last_hour_activity", "is_outside_home_country=", "is_weekend", "hourrisk", "hour="
    ]):
        return "behavior"
    if any(k in s for k in [
        "vendor_category", "vendor_type", "channel=", "card_type", "is_card_present"
    ]):
        return "transaction"
    return "other"

strong["block"] = strong["feature"].map(classify_block)

# ===== 3) УБОРКА ПОЧТИ-ДУБЛЕЙ (особенно для GEO) =====
def collapse_geo_duplicates(frame: pd.DataFrame) -> pd.DataFrame:
    """
    Схлопывает пары вида:
    country=Russia   vs   country=Russia | city=Unknown City
    Оставляем более детальный или более сильный по метрике вариант.
    """
    if frame.empty:
        return frame

    f = frame.copy()
    f["country_key"] = f["feature"].str.extract(r"(country=[^|]+)")
    f["city_key"]    = f["feature"].str.extract(r"(city=[^|]+)")

    # ключ «только страна», чтобы ловить смерть-дубликаты
    only_country = f[~f["country_key"].isna() & f["city_key"].isna()].copy()
    country_city = f[~f["country_key"].isna() & ~f["city_key"].isna()].copy()

    # если есть country=XX и также country=XX|city=Unknown City — оставим тот, у кого метрика выше
    # Возьмём «главную» метрику — используем приоритет: lift > fraud_rate > |spearman|
    def main_metric_row(row):
        if "lift" in row and pd.notna(row["lift"]): return ("lift", row["lift"])
        if "fraud_rate" in row and pd.notna(row["fraud_rate"]): return ("fraud_rate", row["fraud_rate"])
        if "spearman_vs_fraud" in row and pd.notna(row["spearman_vs_fraud"]): return ("spearman", abs(row["spearman_vs_fraud"]))
        return ("none", -np.inf)

    f["_metric_name"], f["_metric_val"] = zip(*f.apply(main_metric_row, axis=1))

    # сгруппируем по country_key и выберем строку с max _metric_val (city-специфику оставим, если она сильнее)
    keep_idx = []
    for ck, grp in f.groupby("country_key", dropna=True):
        # если одинаковые фичи — оставим одну
        grp = grp.sort_values("_metric_val", ascending=False)
        keep_idx.append(grp.index[0])

    # доберём строки без country_key (не гео или гео без пар)
    no_ck = f[f["country_key"].isna()].index.tolist()

    cleaned = f.loc[keep_idx + no_ck].drop(columns=["country_key","city_key","_metric_name","_metric_val"])
    return cleaned

geo_df        = collapse_geo_duplicates(strong[strong["block"] == "geo"])
behavior_df   = strong[strong["block"] == "behavior"].copy()
transaction_df= strong[strong["block"] == "transaction"].copy()

# ===== 4) КОРОТКИЕ ИНСАЙТЫ ДЛЯ НАДПИСЕЙ У БАРОВ =====
def short_insight(label: str, metric: str, value: float) -> str:
    s = label.lower()
    if "is_outside_home_country=1" in s and "is_high_risk_vendor=1" in s:
        return "вне дома + high‑risk"
    if "channel=" in s and "device=" in s:
        return "канал×устройство"
    if "devicefirstseen" in s:
        return "новое устройство"
    if "relativeamountcity" in s:
        return "откл. суммы"
    if "hyperactivityscore" in s:
        return "всплеск активности"
    if "is_card_present=0" in s:
        return "CNP‑риск"
    if "country=" in s and "city=" in s:
        return "гео‑кластер"
    if "country=" in s:
        return "гео‑риск (страна)"
    if "vendor_category" in s or "vendor_type" in s:
        return "категорийный риск"
    return f"{metric}={value:.3f}"

def build_note(value, n, priority_flag):
    note = f"{value:.3f}"
    if n is not None:
        note += f" | n={int(n)}"
        if int(n) < LOW_N: note += " (low N)"
    if priority_flag: note += "  ★"
    return note

def footer_note(metric: str) -> str:
    return (
        "Красный бар + ★ = приоритетная гипотеза.  "
        "Жёлтая обводка = ТОП‑3 по значению метрики.  "
        f"Ось X: {metric}.  Фильтр надёжности: N ≥ {MIN_N}."
    )

def annotate_top3(ax, xvals, yvals, k=3):
    if len(xvals) == 0: return
    order_desc = np.argsort(-xvals)[:min(k, len(xvals))]
    for rank, idx in enumerate(order_desc, start=1):
        ax.annotate(
            f"Топ‑{rank}", xy=(xvals[idx], yvals[idx]),
            xytext=(xvals[idx]*1.02 if xvals[idx] > 0 else 0.02, yvals[idx]+0.22),
            arrowprops=dict(arrowstyle="->", lw=1.2, color="#444"),
            fontsize=10, color="#111"
        )

# ===== 5) УНИВЕРСАЛЬНЫЙ РИСОВАЛЬЩИК СЛАЙДОВ =====
plt.rcParams.update({
    "axes.titlesize": 18, "axes.labelsize": 12, "xtick.labelsize": 10, "ytick.labelsize": 10
})

def plot_section(tbl: pd.DataFrame, title_prefix: str, metric: str, fname: str, top_n=TOP_N):
    if tbl.empty or metric not in tbl or not tbl[metric].notna().any():
        empty_notes.append(f"{title_prefix} — {metric}: пусто → график не строим.")
        return False

    t = (tbl.dropna(subset=[metric])
           .sort_values(metric, ascending=False)
           .drop_duplicates(subset=["feature"], keep="first")
           .head(top_n)
           .copy())
    if t.empty:
        empty_notes.append(f"{title_prefix} — {metric}: после фильтров пусто.")
        return False

    # данные
    y = np.arange(len(t))[::-1]
    x = t[metric].values[::-1]
    labels = t["feature"].values[::-1]
    pr = t["priority"].values[::-1]
    n_vals = t[weight_col].values[::-1] if weight_col else [None]*len(t)
    colors = np.where(pr, "#d62728", "#1f77b4")

    # фигура
    h = max(5.0, 0.80*len(t))
    fig, ax = plt.subplots(figsize=(16, h))
    bars = ax.barh(y, x, color=colors, edgecolor="none")

    # топ‑3 — жёлтая обводка
    top3 = set(np.argsort(-x)[:min(3, len(x))])
    for i, bar in enumerate(bars):
        if i in top3:
            bar.set_edgecolor("#ffbf00")
            bar.set_linewidth(2.4)

    ax.set_yticks(y, labels, fontsize=10)
    ax.set_xlabel(metric)
    ax.set_title(f"{title_prefix}: Top {metric}")

    xmax = float(np.nanmax(x)) if len(x) else 1.0
    ax.set_xlim(0, xmax*1.14 + 1e-6)

    for yi, xi, nn, pflag, lab in zip(y, x, n_vals, pr, labels):
        note = build_note(xi, nn, pflag)
        insight = short_insight(lab, metric, xi)
        ax.text(xi, yi, " " + note + "  —  " + insight, va="center", ha="left", fontsize=10)

    annotate_top3(ax, x, y, k=3)
    fig.text(0.01, 0.01, footer_note(metric), fontsize=9, ha="left", va="bottom", color="#333")

    fig.tight_layout(rect=[0, 0.04, 1, 1])
    out = OUT_DIR / fname
    fig.savefig(out, dpi=240)
    pdf.savefig(fig)
    plt.close(fig)
    print("[saved]", out)
    return True

# ===== 6) СБОРКА СЛАЙДОВ ПО БЛОКАМ =====
subtitle = f"Источник: top_signals_reliable.csv • Топ-{TOP_N} • Фильтр N≥{MIN_N}"

built = 0
for metric in ["lift", "fraud_rate", "spearman_vs_fraud"]:
    built += plot_section(geo_df,        "ГЕО (страна/город)",        metric, f"geo_top_{metric}.png")
    built += plot_section(behavior_df,   "ПОВЕДЕНИЕ (устройства/окна)",metric, f"behavior_top_{metric}.png")
    built += plot_section(transaction_df,"ТРАНЗАКЦИИ (канал/категории)",metric, f"transaction_top_{metric}.png")

# Комбинированный скор — общий «топ-10»
norm_cols, nd = [], strong.copy()
if "lift" in nd:
    v = nd["lift"].clip(lower=0)
    nd["lift_norm"] = (v - v.min())/(v.max()-v.min() + 1e-9); norm_cols.append("lift_norm")
if "fraud_rate" in nd:
    v = nd["fraud_rate"].clip(lower=0)
    nd["fraud_rate_norm"] = (v - v.min())/(v.max()-v.min() + 1e-9); norm_cols.append("fraud_rate_norm")
if "spearman_vs_fraud" in nd:
    v = nd["spearman_vs_fraud"].abs()
    nd["spearman_norm"] = (v - v.min())/(v.max()-v.min() + 1e-9); norm_cols.append("spearman_norm")

if norm_cols:
    nd["score"] = nd[norm_cols].mean(axis=1)
    top = (nd.sort_values("score", ascending=False)
             .drop_duplicates(subset=["feature"])
             .head(TOP_N)
             .copy())

    y = np.arange(len(top))[::-1]
    x = top["score"].values[::-1]
    labels = top["feature"].values[::-1]
    pr = top["priority"].values[::-1]
    n_vals = top[weight_col].values[::-1] if weight_col else [None]*len(top)
    colors = np.where(pr, "#d62728", "#1f77b4")

    h = max(5.0, 0.80*len(top))
    fig, ax = plt.subplots(figsize=(16, h))
    bars = ax.barh(y, x, color=colors, edgecolor="none")

    top3 = set(np.argsort(-x)[:min(3, len(x))])
    for i, bar in enumerate(bars):
        if i in top3:
            bar.set_edgecolor("#ffbf00")
            bar.set_linewidth(2.4)

    ax.set_yticks(y, labels, fontsize=10)
    ax.set_xlabel("Normalized Score (0–1)")
    ax.set_title("СВОДНО: Top‑10 по комбинированному скору")

    xmax = float(np.nanmax(x)) if len(x) else 1.0
    ax.set_xlim(0, xmax*1.14 + 1e-6)

    for yi, xi, nn, pflag, lab in zip(y, x, n_vals, pr, labels):
        note = build_note(xi, nn, pflag)
        insight = short_insight(lab, "score", xi)
        ax.text(xi, yi, " " + note + "  —  " + insight, va="center", ha="left", fontsize=10)

    annotate_top3(ax, x, y, k=3)
    fig.text(0.01, 0.01, footer_note("score"), fontsize=9, ha="left", va="bottom", color="#333")

    fig.tight_layout(rect=[0, 0.04, 1, 1])
    out = OUT_DIR / "overall_top10_combined.png"
    fig.savefig(out, dpi=240)
    pdf.savefig(fig)
    plt.close(fig)
    print("[saved]", out)


[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/geo_top_lift.png
[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/behavior_top_lift.png
[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/transaction_top_lift.png
[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/geo_top_fraud_rate.png
[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/behavior_top_fraud_rate.png
[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/transaction_top_fraud_rate.png
[saved] /content/drive/MyDrive/ИТМО/insights/visuals_final/overall_top10_combined.png


In [5]:
# Клонируем репо в /content/tmp_repo и смотрим структуру
GH_USER = "DANIELVSHVL"
GH_REPO = "ITMO_EXAM"

!rm -rf /content/tmp_repo
!git clone https://github.com/{GH_USER}/{GH_REPO}.git /content/tmp_repo
!tree /content/tmp_repo


Cloning into '/content/tmp_repo'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 18 (delta 1), reused 14 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 9.00 KiB | 9.00 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/bin/bash: line 1: tree: command not found


In [6]:
BRANCH = "feature/data-analysis-answers"

!rm -rf /content/tmp_repo
!git clone -b {BRANCH} https://github.com/{GH_USER}/{GH_REPO}.git /content/tmp_repo
!tree /content/tmp_repo


Cloning into '/content/tmp_repo'...
fatal: Remote branch feature/data-analysis-answers not found in upstream origin
/bin/bash: line 1: tree: command not found


In [ ]:
# ==== НАСТРОЙКИ ====
GH_USER  = "DANIELVSHVL"
GH_REPO  = "ITMO_EXAM"
BRANCH   = "feature/data-analysis-research"

NB_EXPECTED_DIR   = "/content/drive/MyDrive/Colab Notebooks"
ITMO_DIR          = "/content/drive/MyDrive/ITMO"
VISUALS_SUBDIR    = "insights/visuals_final"
TOP_SIGNALS_NAME  = "top_signals_reliable.csv"

USER_EMAIL = "you@example.com"   # замени
USER_NAME  = "Your Name"         # замени

from getpass import getpass
GITHUB_TOKEN = getpass("Введите GitHub Personal Access Token: ")

# ==== КОД ====
import os, shutil, subprocess, pathlib, glob, textwrap
# ФИКС: гарантируем нормальный cwd
os.chdir("/content")

# (монтирование диска на всякий случай)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
except Exception:
    pass

def run(cmd, cwd=None):
    r = subprocess.run(cmd, cwd=cwd, text=True, capture_output=True)
    if r.returncode != 0:
        print(r.stdout); print(r.stderr)
        raise RuntimeError(" ".join(cmd))
    return r.stdout.strip()

def find_one(patterns):
    for pat in patterns:
        hits = glob.glob(pat)
        if hits: return hits[0]
    return None

# ищем файлы
nb_path = find_one([
    os.path.join(NB_EXPECTED_DIR, "02_feature_research.ipynb"),
    os.path.join(NB_EXPECTED_DIR, "*feature*research*.ipynb"),
    os.path.join(NB_EXPECTED_DIR, "*answers*1-11*.ipynb"),
])
if not nb_path:
    raise FileNotFoundError("Не найден ноутбук исследования в 'Colab Notebooks'.")

top_signals_path = find_one([
    os.path.join(ITMO_DIR, "insights", TOP_SIGNALS_NAME),
    os.path.join(ITMO_DIR, "**", TOP_SIGNALS_NAME),
])
visuals_dir = find_one([
    os.path.join(ITMO_DIR, VISUALS_SUBDIR),
    os.path.join(ITMO_DIR, "insights", "visuals_final"),
])

print("[found] notebook:", nb_path)
print("[found] top csv :", top_signals_path or "(skip)")
print("[found] visuals :", visuals_dir or "(skip)")

# готовим локальный репо
workdir = "/content/tmp_repo"
repo_public = f"https://github.com/{GH_USER}/{GH_REPO}.git"
repo_token  = f"https://{GITHUB_TOKEN}@github.com/{GH_USER}/{GH_REPO}.git"

shutil.rmtree(workdir, ignore_errors=True)
run(["git", "clone", repo_public, workdir])

run(["git", "config", "--global", "user.email", USER_EMAIL])
run(["git", "config", "--global", "user.name", USER_NAME])
run(["git", "checkout", "-b", BRANCH], cwd=workdir)

# структура
p_root    = pathlib.Path(workdir, "project1_data_analysis")
p_nb      = p_root / "notebooks"
p_outputs = p_root / "outputs"
p_visuals = p_outputs / "visuals"
for p in [p_nb, p_outputs, p_visuals]:
    p.mkdir(parents=True, exist_ok=True)

# копируем
shutil.copy2(nb_path, p_nb / pathlib.Path(nb_path).name)
if top_signals_path and os.path.isfile(top_signals_path):
    shutil.copy2(top_signals_path, p_outputs / pathlib.Path(top_signals_path).name)
if visuals_dir and os.path.isdir(visuals_dir):
    for f in os.listdir(visuals_dir):
        src = os.path.join(visuals_dir, f)
        if os.path.isfile(src):
            shutil.copy2(src, p_visuals / f)

# коммит и пуш
run(["git", "add", "-A"], cwd=workdir)
run(["git", "commit", "-m", "feat(research): add exploration notebook and selected outputs"], cwd=workdir)
run(["git", "remote", "set-url", "origin", repo_token], cwd=workdir)
run(["git", "push", "-u", "origin", BRANCH], cwd=workdir)
run(["git", "remote", "set-url", "origin", repo_public], cwd=workdir)

print(f"\nГотово → https://github.com/{GH_USER}/{GH_REPO}/tree/{BRANCH}")
